### Load a TCX file, display and analyze

In [10]:
import pandas as pd

# maps
import folium

from utility import load_in_pandas_from_tcx, compute_total_distance

In [11]:
POWER = False
CADENCE = False

In [12]:
F_NAME = "luigi_activity_11448803267.tcx"

df = load_in_pandas_from_tcx(F_NAME, cadence=False, power=False, debug=False)

df['timestamp'] = pd.to_datetime(df['timestamp'])

Exception: time: 2023-06-29T07:31:13.000Z, 'NoneType' object has no attribute 'find'
Exception: time: 2023-06-29T07:31:14.000Z, 'NoneType' object has no attribute 'find'
Exception: time: 2023-06-29T08:15:32.000Z, 'NoneType' object has no attribute 'find'
Exception: time: 2023-06-29T08:27:43.000Z, 'NoneType' object has no attribute 'find'
Exception: time: 2023-06-29T08:27:44.000Z, 'NoneType' object has no attribute 'find'


In [13]:
df.head(10)

,id,timestamp,position_lat,position_long,altitude,distance,speed,heart_rate
0,1,2023-06-29 07:08:51+00:00,41.8832397647202,12.475301558151841,25.2,3.0,11.1,116.0
1,2,2023-06-29 07:08:52+00:00,41.883262479677796,12.475279597565532,12.6,6.1,11.1,116.0
2,3,2023-06-29 07:08:57+00:00,41.883373288437724,12.475134255364537,12.6,23.4,13.3,115.0
3,4,2023-06-29 07:09:03+00:00,41.88352935947478,12.474955217912793,12.4,46.2,13.3,112.0
4,5,2023-06-29 07:09:04+00:00,41.883554589003325,12.474927809089422,12.4,49.8,13.3,112.0
5,6,2023-06-29 07:09:10+00:00,41.88367260619998,12.474807109683752,12.2,66.4,8.4,111.0
6,7,2023-06-29 07:09:12+00:00,41.88371107913554,12.474840972572565,12.4,71.6,10.8,110.0
7,8,2023-06-29 07:09:13+00:00,41.88372876495123,12.474874584004283,12.4,75.0,10.8,109.0
8,9,2023-06-29 07:09:14+00:00,41.88374628312886,12.474911883473396,12.4,78.7,10.8,108.0
9,10,2023-06-29 07:09:20+00:00,41.883869832381606,12.475104751065373,12.0,99.8,12.9,105.0


#### Compute Statistics

In [14]:
all_cols = df.columns
exc_cols = ['id','timestamp','position_lat','position_long']
allow_cols = list(set(all_cols) - set(exc_cols))

# ignored first row, with counts
pd.set_option('display.precision', 1)

df[allow_cols].describe()[1:]

,distance,speed,heart_rate,altitude
mean,30461.1,23.5,135.4,10.2
std,18820.5,6.8,10.4,5.1
min,3.0,0.0,90.0,-1.2
25%,12985.9,20.8,130.0,7.8
50%,29897.7,25.7,137.0,12.6
75%,48443.0,28.1,143.0,13.8
max,59817.2,40.5,160.0,25.2


In [15]:
# remove when lat, long are nan
df = df.dropna(ignore_index=True)

# time elapsed
time_diff = df['timestamp'].iloc[-1] - df['timestamp'].iloc[0]

print(f"Elapsed time: {time_diff}")

# tot km
tot_km = round(compute_total_distance(df)/1000., 2)

print(f"Tot km. travelled: {tot_km}")

# va aggiunto il metabolismo basale (sono solo le calorie portate ai pedali)
# enhanced using eff factor (default = 0.25)
# we're assuming time between points is 1 sec.
if POWER:
    print(f"Tot active cal. consumed: {compute_energy_consumed(df, 'power'):.0f} kcal.")

Elapsed time: 0 days 02:43:37
Tot km. travelled: 59.82


In [16]:
# remove eventually rows where lat, lon are nan
df = df.dropna(ignore_index=True)

m = folium.Map(location=[df['position_lat'][1], df['position_long'][1]], zoom_start=15)

# to create the popup
cols_list = ['position_lat', 'position_long', 'altitude', 'speed', 'heart_rate']
if POWER:
    cols_list.append('power')
if CADENCE:
    cols_list.append('cadence')
    
for i, row in df[cols_list].iterrows():
    f_str = f"<b>Altitude: {row['altitude']:.1f} m.</b><br><b>Speed: {row['speed']:.1f} kmh</b><br><b>HR: {row['heart_rate']:.0f}</b>"

    if POWER:
        f_str += f"<br><b>Power: {row['power']} w."
    if CADENCE:
        f_str += f"<br><b>Cadence: {row['cadence']}"
        
    popup_html = folium.Popup(f_str, max_width=200)
    
    folium.CircleMarker(location=[row['position_lat'], row['position_long']],
                        popup=popup_html,
                        radius=3).add_to(m)

n_rows = i

# add start (green) end (red)
folium.CircleMarker(
    location=[df['position_lat'][0], df['position_long'][0]],  # Latitude and Longitude
    radius=5,  # Size of the circle
    color='green',  # Border color of the circle
    fill=True,
    fill_color='green'  # Fill color of the circle
).add_to(m)
folium.CircleMarker(
    location=[df['position_lat'][n_rows], df['position_long'][n_rows]],  # Latitude and Longitude
    radius=5,  # Size of the circle
    color='red',  # Border color of the circle
    fill=True,
    fill_color='red'  # Fill color of the circle
).add_to(m)

# display the map
m